# BADASS Maximum-Likelihood fitting 

### Remington O. Sexton 

In [ ]:
# % matplotlib notebook
% matplotlib inline

import glob
from time import clock
from os import path
import os
import shutil
from astropy.io import fits
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
from numpy.polynomial import legendre, hermite
from scipy import linalg, special, fftpack
import sys
import emcee
# import scipy.optimize as op
import matplotlib.gridspec as gridspec
from numpy import linspace, meshgrid
from scipy.interpolate import griddata, interp1d
from matplotlib import cm
import psutil
import time
from scipy.stats import kde, norm
import matplotlib.gridspec as gridspec
from scipy.integrate import simps
import datetime
from astropy.stats import mad_std
import natsort
import badass_sdss_type_1_v3 as badass


In [ ]:
### spec_dir = '/Users/rem/BADASS_SDSS_Type_1/S18_SDSS/'
spec_dir = '/Users/rem/Bennert_2018/refit/'
ppxf_dir = '/Users/rem/BADASS_SDSS_Type_1/badass_data_files/'
temp_dir = ppxf_dir+'final_library'

# Fitting parameters
fitreg = (4400,5800) # Fitting region; MILES Stellar Library=(3540,7409); Indo-US Library=(3460,9464)

# MCMC algorithm parameters
auto_stop = 1 # Automatic stop when 1% threshold is reached
auto_stop_thresh = 1.0 # percentage between checking iterations below which we can automatically stop MCMC
burn_in = 500 # take last N iterations
write_iter = 100 # check every N number of iterations
# min_iter = 1500 # minimum number of iterations before checking; default 2*burn_in
max_iter = 5000 # max number of iterations before stopping

# Get full list of spectrum folders; these will be the working directories
spec_loc = natsort.natsorted( glob.glob(spec_dir+'J*') )

# for i in range(0,len(spec_loc),1):
for i in range(0,1,1):
    work_dir = spec_loc[i]+'/'
    run_dir,prev_dir = badass.setup_dirs(work_dir)
    file = glob.glob(work_dir+'*.fits')[0]
    # Uncomment below line to use previously-chosen templates
#     temp_dir = spec_loc[i]+'/'+'templates_final'
    
    print('\n Starting object %s...' % (spec_loc[i][-19:]) )
    # Open the spectrum to get the redshift 
    hdulist = fits.open(file)
    header = hdulist[0].header
#     fitreg = (4400,5800) # Fitting region; MILES Stellar Library: (3540,7409)

    # Prepare SDSS spectrum for fitting
    lam_gal,galaxy,templates,noise,fwhm_gal,velscale,dv,temp_list,z,ebv = badass.sdss_prepare(file,fitreg,temp_dir,run_dir)
    print(' z = %0.4f' % z)
    print(' E(B-V) =  %0.4f' % ebv)
    print(' velscale = %0.4f' % velscale)

    # Create FeII templates
    na_feii_template,br_feii_template = badass.initialize_feii(lam_gal,velscale)
    
    # Import galaxy template
#     gal_temp = badass.galaxy_template(lam_gal,age=15)
    gal_temp = badass.galaxy_template(lam_gal)
#     if 1: sys.exit()

    ###########################################################################################################
    # If previous MCMC directory exists, get the last MCMC chain values to use as initial parameters, 
    # otherwise perform max likelihood fitting for initial parameters
#     if prev_dir is not None:
#         if os.path.exists(prev_dir+'MCMC_chain.csv')==True:
#             try:
#                 print(' Previous MCMC folder exists.  Getting initial parameters from previous MCMC chain...')
#                 with open(prev_dir+'MCMC_chain.csv','rb') as f: lines = f.readlines()
#                 line1 = np.array(lines[-2].split(','))[1:]
#                 line1 = line1.astype(np.float)
#                 if len(line1)==17: # if outflow 
#                     print(" Using outflow model...")
#                     final_model = 'outflow'
#                     init_params = line1
#                     run_maxlike = 0
#                     param_names,param_labels,params,param_limits,npix,ntemp,temp_fft,npad = badass.initialize_mcmc_final_outflows(lam_gal,galaxy,templates,velscale)
#                     args = (lam_gal,fwhm_gal,na_feii_template,br_feii_template,temp_fft,npad,velscale,npix,dv,galaxy,noise) # these things don't change
#                 elif len(line1)==14: # if no outflow
#                     print(" Using NO outflow model...")
#                     final_model = 'NO_outflow'
#                     init_params = line1
#                     run_maxlike = 0
#                     param_names,param_labels,params,param_limits,npix,ntemp,temp_fft,npad = badass.initialize_mcmc_final_no_outflows(lam_gal,galaxy,templates,velscale)
#                     args = (lam_gal,fwhm_gal,na_feii_template,br_feii_template,temp_fft,npad,velscale,npix,dv,galaxy,noise) # these things don't change
#             except ValueError:
#                 print(' WARNING: previous MCMC_chain.csv is empty! ')
#                 run_maxlike = 1
#         elif os.path.exists(prev_dir+'MCMC_chain.csv')==False:
#             print(' WARNING: Could not find previous MCMC_chain.csv!')
#             run_maxlike = 1
#     elif prev_dir is None:
#         print(' Previous fits do not exist.')
#         run_maxlike=1
    run_maxlike = 1
    ###########################################################################################################
        
    if run_maxlike==1:
        print(' Performing Max. Likelihood calculation for outflow model...')
        param_labels,params,param_limits,param_init = badass.initialize_mcmc_init_outflows(lam_gal,galaxy,velscale)
        # Initialize function arguments
        args = (lam_gal,fwhm_gal,na_feii_template,br_feii_template,gal_temp,velscale,galaxy,noise,run_dir) # these things don't change
        result_outflows = badass.max_likelihood_outflows(params,param_limits,args)
        par_best_outflows = result_outflows["x"]
        outflow_std = badass.init_outflow_model(par_best_outflows,lam_gal,fwhm_gal,na_feii_template,br_feii_template,gal_temp,\
                                                                     velscale,galaxy,run_dir,output_model=True)
                
        print(' Outflow Resid. STD = %0.3f' % outflow_std)

        ###########################################################################################################
#         print(' Performing Max. Likelihood calculation for NO outflow model...')
#         param_labels,params,param_limits,param_init = badass.initialize_mcmc_init_no_outflows(lam_gal,galaxy,velscale)
#         # Initialize function arguments
#         args = (lam_gal,fwhm_gal,na_feii_template,br_feii_template,gal_temp,velscale,galaxy,noise,run_dir) # these things don't change
#         result_no_outflows = badass.max_likelihood_no_outflows(params,param_limits,args)
#         par_best_no_outflows = result_no_outflows["x"]
#         no_outflow_std = badass.init_no_outflow_model(par_best_no_outflows,lam_gal,fwhm_gal,na_feii_template,br_feii_template,gal_temp,\
#                                                         velscale,galaxy,run_dir,output_model=True)
#         print(' NO Outflow Resid. STD = %0.3f' % no_outflow_std)
            
        # Determine which final model to use 
        # If outflow FWHM > core FWHM, and 
        # outflow VOFF < core VOFF (blueshifted), and 
        # outflow amplitude > outflow STD, then use the outflow final model
    
#         print (par_best_outflows[7]>(par_best_outflows[4])) # fwhm outflow > fwhm core
#         print (par_best_outflows[8] < par_best_outflows[5]) # voff outflow < voff core
#         print (par_best_outflows[6]>(1.0*outflow_std)) # outflow amp > noise
#         print (par_best_outflows[3] > par_best_outflows[6])# amplitude core > amplitude outflow
            
        if 1:#(par_best_outflows[7]>(par_best_outflows[4])) and (par_best_outflows[8] < (par_best_outflows[5]))\
#             and (par_best_outflows[6]>(1.0*outflow_std)) and (par_best_outflows[3] > par_best_outflows[6]):
            print(" Using outflow model...")
            final_model = 'outflow'
            init_params = np.concatenate(([150.0,150.0,0.0,0.0],par_best_outflows[1:]))
            param_names,param_labels,params,param_limits,npix,ntemp,temp_fft,npad = badass.initialize_mcmc_final_outflows(lam_gal,galaxy,templates,velscale)
            args = (lam_gal,fwhm_gal,na_feii_template,br_feii_template,temp_fft,npad,velscale,npix,dv,galaxy,noise) # these things don't change
#         else: 
#             print(" Using NO outflow model...")
#             final_model = 'NO_outflow'
#             init_params = np.concatenate(([150.0,150.0,0.0,0.0],par_best_no_outflows[1:]))
#             param_names,param_labels,params,param_limits,npix,ntemp,temp_fft,npad = badass.initialize_mcmc_final_no_outflows(lam_gal,galaxy,templates,velscale)
#             args = (lam_gal,fwhm_gal,na_feii_template,br_feii_template,temp_fft,npad,velscale,npix,dv,galaxy,noise) # these things don't change
                
    for i in range(0,len(init_params),1):
        print(' %s = %0.2f' % (param_names[i],init_params[i]))
    
#     print param_names 
#     if 1: sys.exit()
        
    #######################################################################################################
    print(' Performing MCMC iterations...')
    # Emcee
    ndim, nwalkers = len(init_params), 50 # minimum walkers = 2*len(params)
        
    # initial parameters Maximum Likelihood best-fit
    pos = [init_params + 1e-2*np.random.randn(ndim) for i in range(nwalkers)]
        
    # initial parameters from an initial run using MCMC
    sampler = emcee.EnsembleSampler(nwalkers, ndim, badass.lnprob, args=(param_limits,lam_gal,fwhm_gal,na_feii_template,br_feii_template,\
                                                                        temp_fft,npad,velscale,npix,dv,galaxy,noise,temp_list,run_dir,\
                                                                        final_model),threads=10)
    # Open a file to save progress of chain
    f = open(run_dir+'MCMC_chain.csv','w')
    param_string = ', '.join(str(e) for e in param_names)
    f.write('# iter, ' + param_string) # Write initial parameters
    best_str = ', '.join(str(e) for e in init_params)
    f.write('\n 0, '+best_str)
    f.close()
    start_time = time.time() # start timer
    for k, result in enumerate(sampler.sample(pos, iterations=max_iter)):
    #     print k
        if (k>1) and ((k+1) % write_iter == 0):
                print (k+1)
    #         time_rem = 100./((float(k)*100. / max_iter)/(time.time() - start_time))
    #         print('--- Time Remaining: %s --- %0.2f%% Complete ---' % ((str(datetime.timedelta(seconds=(time_rem)-(time.time() - start_time)))),(float(k)*100 / max_iter)) )
        if ((k+1) % write_iter == 0) and ((k+1)>=burn_in): # Write every [write_iter] iteration
            flat = sampler.chain[:,k+1-burn_in:k+1,:].reshape((-1, ndim))
            print np.shape(flat), k+1-burn_in, k+1
            best = []
            for pp in range(0,np.shape(flat)[1],1):
                data = (flat[:,pp])
                mu, std = norm.fit(data)
                best.append(mu)
#             if 1: sys.exit()
            f = open(run_dir+'MCMC_chain.csv','a')
    #         print best_str
            best_str = ', '.join(str(e) for e in best)
            f.write('\n'+str(k+1)+', '+best_str)
            f.close()
        if ((k+1) % write_iter == 0) and ((k+1)>=2*burn_in) and (auto_stop==1): # Auto Stop: Every 500th iteration, compare the last step
            # Compare the last two lines
            with open(run_dir+'MCMC_chain.csv','rb') as f: lines = f.readlines()
            line2 = np.array(lines[-1].split(','))[1:]
            line2 = line2.astype(np.float)
            line1 = np.array(lines[-2].split(','))[1:]
            line1 = line1.astype(np.float)
            lim = np.abs(np.abs(line1-line2)/np.mean([line1,line2],axis=0)*100.)
            if (len(param_names)==17): # outflow 
                check_par = np.array([param_names[param_names.index('vel')],param_names[param_names.index('vel_disp')],\
                                      param_names[param_names.index('oiii5007_core_amp')],param_names[param_names.index('oiii5007_core_fwhm')],\
                                      param_names[param_names.index('oiii5007_outflow_amp')],param_names[param_names.index('oiii5007_outflow_fwhm')],\
                                      param_names[param_names.index('br_Hb_amp')],param_names[param_names.index('br_Hb_fwhm')]])
                check_arr = np.array([lim[param_names.index('vel')],lim[param_names.index('vel_disp')],\
                                      lim[param_names.index('oiii5007_core_amp')],lim[param_names.index('oiii5007_core_fwhm')],\
                                      lim[param_names.index('oiii5007_outflow_amp')],lim[param_names.index('oiii5007_outflow_fwhm')],\
                                      lim[param_names.index('br_Hb_amp')],lim[param_names.index('br_Hb_fwhm')]])
                print check_par, check_arr
                for p,l in enumerate(check_arr): 
                    if l>auto_stop_thresh:
                        print(' %s has not converged! (%0.2f)' % (check_par[p],l))
                if (all(check_arr<auto_stop_thresh)==True) and (lim[param_names.index('br_Hb_fwhm')]<0.1): 
                    print("--- %s seconds ---" % (time.time() - start_time))
                    break
            elif (len(param_names)==14): # no outflow
                check_par = np.array([param_names[param_names.index('vel')],param_names[param_names.index('vel_disp')],\
                                      param_names[param_names.index('oiii5007_core_amp')],param_names[param_names.index('oiii5007_core_fwhm')],\
                                      param_names[param_names.index('br_Hb_amp')],param_names[param_names.index('br_Hb_fwhm')]])
                check_arr = np.array([lim[param_names.index('vel')],lim[param_names.index('vel_disp')],\
                                      lim[param_names.index('oiii5007_core_amp')],lim[param_names.index('oiii5007_core_fwhm')],\
                                      lim[param_names.index('br_Hb_amp')],lim[param_names.index('br_Hb_fwhm')]])
                print check_par,check_arr
                for p,l in enumerate(check_arr): 
                    if l>auto_stop_thresh:
                        print(' %s has not converged! (%0.2f)' % (check_par[p],l))
                if (all(check_arr<auto_stop_thresh)==True) and (lim[param_names.index('br_Hb_fwhm')]<0.10): 
                    print("--- %s seconds ---" % (time.time() - start_time))
                    break

    ###################################################################################
#     if 1: sys.exit()
    ###########################################################################################################
    print(' Making plots!')
    # burnin=max_iter-(int(round(max_iter*0.15)))
    nfinal = (k+1)
    # Prune the sampler chain of excess zero-arrays
    sampler_chain = sampler.chain[:,:nfinal,:]
    burnin = nfinal-burn_in # Take last 500 steps
    par_figname,par_best,par_sig_low,par_sig_upp = badass.param_plots(params,param_labels,burnin,sampler_chain,run_dir,model_type=final_model)
    eline_flux_figname,eline_flux_best,eline_flux_sig_low,elin_flux_sig_upp = badass.emline_flux_plots(run_dir,model_type=final_model)
    eline_lum_figname,eline_lum_best,eline_lum_sig_low,elin_lum_sig_upp = badass.flux2lum(eline_flux_figname,eline_flux_best,\
                                                                                              eline_flux_sig_low,elin_flux_sig_upp,z,run_dir,H0=71.0,Om0=0.27)
    # Concatenate all final best-fit values
    figname  = np.concatenate((par_figname,eline_flux_figname,eline_lum_figname))
    best_val = np.concatenate((par_best,eline_flux_best,eline_lum_best))
    sig_low  = np.concatenate((par_sig_low,eline_flux_sig_low,eline_lum_sig_low))
    sig_upp  = np.concatenate((par_sig_upp,elin_flux_sig_upp,elin_lum_sig_upp))
        
    badass.write_fits_table(figname,best_val,sig_low,sig_upp,run_dir,model_type=final_model)
    badass.write_MCMC_data(sampler_chain,run_dir,model_type=final_model)
        
    # Get best-fit model
    # Re-compile templates, this time using all 600+ templates from the Indo-US Coude´ Library of Stellar Templates
    # temp_dir = ppxf_dir+'Indo_US_models'
    if final_model=='outflow':
        best_model = badass.final_outflow_model(par_best,lam_gal,fwhm_gal,na_feii_template,br_feii_template,\
                                                    temp_fft,npad,velscale,npix,dv,galaxy,temp_list,run_dir,output_model=True)
    elif final_model=='NO_outflow':
        best_model = badass.final_no_outflow_model(par_best,lam_gal,fwhm_gal,na_feii_template,br_feii_template,\
                                                    temp_fft,npad,velscale,npix,dv,galaxy,temp_list,run_dir,output_model=True)
    badass.plot_best_model(lam_gal,galaxy,best_model,run_dir,model_type=final_model) 
        
    #######################################################################################################################
            